In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 13 20:45:10 2017

@author: merry jane

调用训练好的lstm模型 预测句子中每个词语的概率和ppl
2、输入句子，返回句子词语 概率 和ppl困惑度 概率值的几何平均取倒
"""
from keras.models import load_model
import jieba
import re
import data_helper
import numpy as np
import jieba.posseg


def get_words_prob(sentence):
    '''
    预测句子每个词条件概率，该句子的通顺性值=1/词语概率乘积
    '''
    
    sentence="".join([x.strip() for x in re.findall('(.*?)[|,|;|?|.|!|:|，|。|、|~|；|！|？|"|“|”|】|【|《|》|<|>|&|%|$|(|)|（|）|^|#|@]', sentence+',')])
    sen_words=jieba.lcut(sentence)
    p = re.compile(r'\d+')
    sen_words=[word for word in   filter(lambda x:len(re.findall(p,x))==0 ,sen_words)] 
    word_prob={}
    word_prob_list=[]
    i=0
    ok_w=[]
    for w in sen_words:
        if i >=len(sen_words):
            break
            
        next_w=sen_words[i]
        
        input_words=["的"]*(maxlen-i)+ok_w[-maxlen:]
        x=np.zeros((1,maxlen,len(char_indices)))
    
        for t,char in enumerate(input_words):
            try:
                x[0,t,char_indices[char]]=1.
            except:
                continue

        i+=1
   
   

        preds=model.predict(x,verbose=0)[0]
        try:
            word_prob[next_w]=preds[char_indices[next_w]]
        except:
            word_prob[next_w]=0.0
        word_prob_list.append( word_prob[next_w])
        
        ok_w.append(w)
    
    
    return word_prob,{"ppl":(np.array(word_prob_list[1:]).prod())**(-1/len(word_prob_list))}

def sample(preds,temperature=1.0):
    preds=np.asarray(preds).astype("float64")
    scaled_preds=preds**(1/temperature)
    preds=scaled_preds/np.sum(scaled_preds)
    probas=np.random.multinomial(1,preds,1)
    return np.argmax(probas)

def get_next_words(sentence,ntimes=20):
#    sentence='讲究简单，朴素无华，细腻光滑工艺外壳，光泽温和，质感醉厚，'
    generated=""
    generated_list=["(",")","/","）"]
    sentence="".join([x.strip() for x in re.findall('(.*?)[|,|;|?|.|!|:|，|。|、|~|；|！|？|"|“|”|】|【|《|》|<|>|&|%|$|(|)|（|）|^|#|@]', sentence+',')])
    sen_words=jieba.lcut(sentence)[-maxlen:]
    p = re.compile(r'\d+')
    sen_words=[word for word in   filter(lambda x:len(re.findall(p,x))==0 ,sen_words)] 

    input_words=sen_words
    x=np.zeros((1,maxlen,len(char_indices)))

    current_w=sen_words[-1]
    for w ,f in jieba.posseg.cut(current_w):
        current_w_flag=f
#        print (w,f)    
    
    
    for i in range(ntimes):
  

        for t,char in enumerate(input_words):
            try:
                x[0,t,char_indices[char]]=1.
            except:
                continue
        preds=model.predict(x,verbose=0)[0]
        next_w=indices_char[sample(preds,0.8)]
        
        for w ,f in jieba.posseg.cut(next_w):
            next_w_flag=f
#            print (w,f)

        if next_w not in generated_list and (current_w_flag!="f" or next_w_flag!="f") and (current_w_flag!="c" or next_w_flag!="c"):
            generated_list.append(next_w)
            generated+=next_w
            input_words=input_words[1:]+[next_w]
            
            current_w=next_w
            current_w_flag=next_w_flag
        else:
            pass
    
    return generated

def check_structure():
    
    pass 


if __name__=='__main__':
    maxlen=10
    
    model = load_model('./lstm_model_gen_word_epoch_20.h5')
    _, char_indices, indices_char = data_helper.load_preprocess()
#    res=get_words_prob('讲究简单，朴素无华，细腻光滑工艺外壳，光泽温和，质感醉厚，')
#    print (res)
    sentence="赣南脐橙更好吃 果肉饱满 口感酸甜"
    res=get_next_words(sentence,ntimes=40)
    
    print (sentence)
    print (res)
    
    
    
    '''
    返回结果：
    ({'光泽': 4.7096591,
  '光滑': 5.9352827,
  '外壳': 2.7460401,
  '工艺': 21.515692,
  '朴素无华': 0.16896033,
  '温和': 3.5096765,
  '简单': 14.434443,
  '细腻': 26.224731,
  '讲究': 0.081697658,
  '质感': 19.405516,
  '醉厚': 0.0076282052},
   2.1476565351778878)
    '''


Using TensorFlow backend.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.645 seconds.
Prefix dict has been built succesfully.


赣南脐橙更好吃 果肉饱满 口感酸甜
可口水果 小和最或


In [2]:
res=get_next_words(sentence,ntimes=40)
print (sentence)
print (res)


赣南脐橙更好吃 果肉饱满 口感酸甜
可口水果 小也肉是因此作用食用了时和功效含量或


In [5]:
print (sentence)
print (sen_words)


讲究简单朴素无华细腻光滑工艺外壳光泽温和质感醉厚
['讲究', '简单', '朴素无华', '细腻', '光滑', '工艺', '外壳', '光泽', '温和', '质感', '醉厚']


In [6]:
input_words=sen_words
x=np.zeros((1,maxlen,len(char_indices)))

for t,char in enumerate(input_words):
    try:
        x[0,t,char_indices[char]]=1.
    except:
        continue

In [8]:
print (x)
print (char)
print (char_indices[char])



[[[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]]
醉厚
6500


In [10]:
preds=model.predict(x,verbose=0)[0]
print (preds)
print (len(preds))

[  1.68687679e-04   1.31132481e-06   1.31139996e-06 ...,   6.43732492e-04
   1.28586112e-06   1.30233661e-06]
9000


In [15]:
def sample(preds,temperature=1.0):
    preds=np.asarray(preds).astype("float64")
    scaled_preds=preds**(1/temperature)
    preds=scaled_preds/np.sum(scaled_preds)
    probas=np.random.multinomial(1,preds,1)
    return np.argmax(probas)

In [16]:
sample(preds)

6960

In [17]:
indices_char[6960]

'垂直'

In [23]:
def get_next_words(sentence,ntimes=20):
#    sentence='讲究简单，朴素无华，细腻光滑工艺外壳，光泽温和，质感醉厚，'
    generated=""
    sentence="".join([x.strip() for x in re.findall('(.*?)[|,|;|?|.|!|:|，|。|、|~|；|！|？|"|“|”|】|【|《|》|<|>|&|%|$|(|)|（|）|^|#|@]', sentence+',')])
    sen_words=jieba.lcut(sentence)
    p = re.compile(r'\d+')
    sen_words=[word for word in   filter(lambda x:len(re.findall(p,x))==0 ,sen_words)] 

    input_words=sen_words
    x=np.zeros((1,maxlen,len(char_indices)))


    
    for i in range(ntimes):
        for t,char in enumerate(input_words):
            try:
                x[0,t,char_indices[char]]=1.
            except:
                continue
        preds=model.predict(x,verbose=0)[0]
        next_w=indices_char[sample(preds,0.8)]
        generated+=next_w
        input_words=input_words[1:]+[next_w]
    
    return generated

In [24]:
get_next_words(sentence,ntimes=20)

'神秘自然复古鞋帮的组合内存创意运动优势巧克力更的款式舒适百变采用的让孩子'

In [11]:
for w ,f in jieba.posseg.cut("或但是因此可所以尤其"):
    print (w,f)

或 c
但是 c
因此 c
可 v
所以 c
尤其 d
